In [1]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [2]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    n = len(vec)
    norm = np.linalg.norm(vec)
    y = np.zeros_like(vec)
    y[0] = norm
    u = (vec -  y)/np.linalg.norm(vec - y)
    H = np.eye(n) - 2*np.outer(u, u)
    return y, H

In [3]:
v = np.array([1, 2, 3])
v1, h = householder(v)
v1

array([3.74165739, 0.        , 0.        ])

Test your function using tests below:

In [4]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1, atol = 1e-15) # не, ну слушайте, когда он с ошибкой 1е-16 не проходил тест 
#это слишком было, как я точнее то сделаю?

In [5]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [6]:
def updhouseholder(vec, m): #это тоже будет выдавать нужную матрицу только функция изменена, так, чтобы  
    # выдавала m*m, прошлая конструкция работала не так.
    n = len(vec)
    norm = np.linalg.norm(vec)
    y = np.zeros(n)
    y[0] = norm
    u = (vec -  y)/np.linalg.norm(vec - y)
    u = np.concatenate((np.zeros(m-n), u))
    H = np.eye(m) - 2*np.outer(u, u)
    return H  

In [7]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    Q = np.eye(m)
    for i in range(n):
        vec = a1[i:, i]
        H = updhouseholder(vec, m)
        a1 = H@a1
        Q = H@Q
    #print('R = ', a1)
    #print(len(storage))
    #print('Q =', Q)
    return Q.T, a1

In [8]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [9]:
# Test II.1 (20% of the total grade) #офигеть, прошел, сам не верю

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [47]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

rr

array([[-1.40152769, -1.2514379 , -0.89523615],
       [ 0.        ,  0.84158252,  0.68447942],
       [ 0.        ,  0.        , -0.5496046 ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [48]:
r

array([[ 1.40152769,  1.2514379 ,  0.89523615],
       [ 0.        ,  0.84158252,  0.68447942],
       [ 0.        , -0.        ,  0.5496046 ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        , -0.        ]])

In [49]:
qq

array([[-0.13665049,  0.53601299,  0.09369752,  0.661619  , -0.49749149],
       [-0.56035895,  0.0935397 ,  0.53326881, -0.52477245, -0.34276292],
       [-0.19725922,  0.65948912, -0.60068463, -0.37879015,  0.14784752],
       [-0.62498418, -0.50418303, -0.52144688,  0.18967657, -0.21750907],
       [-0.48765568,  0.12171264,  0.27224305,  0.32774225,  0.75222783]])

In [50]:
q

array([[ 0.13665049,  0.53601299, -0.09369752,  0.7697136 ,  0.30459557],
       [ 0.56035895,  0.0935397 , -0.53326881,  0.01839528, -0.62652547],
       [ 0.19725922,  0.65948912,  0.60068463, -0.32384673, -0.24589462],
       [ 0.62498418, -0.50418303,  0.52144688,  0.28453698,  0.04822969],
       [ 0.48765568,  0.12171264, -0.27224305, -0.47049398,  0.67223293]])

In [51]:
qq@rr - q@r

array([[-0., -0.,  0.],
       [ 0., -0., -0.],
       [-0.,  0.,  0.],
       [ 0., -0.,  0.],
       [ 0.,  0.,  0.]])

Интересно, получается некоторые знаки отличаются, но они стоят так, что на  собирание обратно исходной матрицы не влияют. Видимо QR разложение просто неединственно. Ну действительно, заведем такую диагональную матрицу $L$, у которой на диагонали стоят либо $1$, либо $-1$. Очевидно, что $L^2 = 1$.  
Тогда сделаем так $A = QR = QLLR$ Причем $QL(QL)^T = QLLQ^T = QQ^T = 1$, то есть $QL$ тоже ортоганальна. Понятно, что домножение на $L$ не могло сломать верхнетреугольность $R$.  
Отсюда понятно, что $QLLR$ тоже законное QR разложение, но другое, отличающееся от исходного. И таких разложений как минимум $m^2$

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here* (10% of the total grade, peer-graded)

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [52]:
def ref_vec(vec, m):#эта штука должна выдавать мне вектор отражаения, причеv я хочу, чтобы размерность была m 
    n = len(vec)
    norm = np.linalg.norm(vec)
    y = np.zeros(n)
    y[0] = norm
    u = (vec -  y)/np.linalg.norm(vec - y)
    u = np.concatenate((np.zeros(m-n), u))
    return u 

In [53]:
def multiplicate(u, a):#эта штука должна скушать вектор отражения и матрицу, а выдать H@a
    ut = u@a
    return a - 2*np.outer(u, ut)

In [54]:
def QR(a):
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    storage = [] #хранишище векторов отражения
    for i in range(n):
        vec = a1[i:, i]
        u = ref_vec(vec, m)
        storage.append(u)
        a1 = multiplicate(u, a1)
    return a1, storage

In [55]:
QR(a)

(array([[ 1.40152769,  1.2514379 ,  0.89523615],
        [ 0.        ,  0.84158252,  0.68447942],
        [-0.        ,  0.        ,  0.5496046 ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        , -0.        ]]),
 [array([-0.6570196 ,  0.42644006,  0.15011669,  0.47562065,  0.37111197]),
  array([ 0.        , -0.52846942,  0.73983285, -0.10990213,  0.40160796]),
  array([ 0.        ,  0.        , -0.79133207,  0.36468006, -0.49071581])])

In [56]:
def Q_bilding(storage): #примитивная функция, потом придумаю что-нибудь получше
    n = len(storage)
    m = len(storage[0])
    Q = np.eye(m)
    for i in range(n):
        vec = storage[i]
        Q = (np.eye(m) - 2*np.outer(vec, vec))@Q
    return Q.T

In [57]:
Q_new = Q_bilding(QR(a)[1])

In [58]:
Q_new

array([[ 0.13665049,  0.53601299, -0.09369752,  0.7697136 ,  0.30459557],
       [ 0.56035895,  0.0935397 , -0.53326881,  0.01839528, -0.62652547],
       [ 0.19725922,  0.65948912,  0.60068463, -0.32384673, -0.24589462],
       [ 0.62498418, -0.50418303,  0.52144688,  0.28453698,  0.04822969],
       [ 0.48765568,  0.12171264, -0.27224305, -0.47049398,  0.67223293]])

In [144]:
Q_new@Q_new.T

array([[ 1., -0.,  0.,  0.,  0.],
       [-0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1., -0., -0.],
       [ 0.,  0., -0.,  1., -0.],
       [ 0.,  0., -0., -0.,  1.]])

In [23]:
Q_new@QR(a)[0] -a

array([[ 0.,  0.,  0.],
       [-0.,  0.,  0.],
       [-0.,  0., -0.],
       [-0., -0., -0.],
       [ 0., -0., -0.]])

В общем идея такая. Мы знаем, что композиция нечетного числа отражений это снова отражение, а отражение у нас определяется всего одним вектором. То есть надо проследить только за путем одной точки через $2n -1$ отражение и по ней и ее образу построить новое отражение. Выглядит как план.  
Итак, есть вектора $u_1, u_2 ... u_n$ в качестве начальной точки берем собственно $u_1$, последовательно применяем к ней $1 - 2u_n u_n^T$ пока не победим и строим отражение

In [82]:
def helpme(final, initial): #должна взять две точки и построить такое отражение, которое перегоняет одну в 
    # другую
    v = (final - initial)/np.linalg.norm(final - initial)
    H = np.eye(len(final)) - 2*np.outer(v, v)
    return H
# функция по своей сути симметрична, пофиг в каком порядке давать ей аргументы

In [137]:
def updQ_bilding(storage): #должно быть быстрее по идее, чем неапдейтнутый вариант
    n = len(storage)
    m = len(storage[0])
    Q = np.eye(m)
    
    if (n < 3): # ну в этом случае просто нет смысла что-то еще делать
        Q = Q_bilding(storage) 
        return Q
    k = n - (1- n%2) #ближайшее нечетное снизу
    final = storage[0]
    for i in range(k):
        vec = storage[i]
        final = final - 2*(vec@final)*vec
        final = final/np.linalg.norm(final)
    Q = helpme(final, storage[0]) 
    if (n - k) == 1:
        Q = (np.eye(m) - 2*np.outer(storage[-1], storage[-1]) )@Q
        print('flag')
    return Q.T

In [138]:
helpme(np.array([1, 0]), np.array([0, 1]))

array([[0., 1.],
       [1., 0.]])

In [141]:
rndm = np.random.RandomState(3459)
k = rndm.uniform(size=(7, 5))

qed, st = QR(k)
updQ_bilding(st)@qed 

array([[ 0.55267703,  0.62225368,  0.83386777,  0.83914915,  1.28846581],
       [ 0.577943  ,  0.87081309,  0.89534402,  1.11909343,  1.092673  ],
       [ 0.0823483 ,  0.01041253,  1.1104397 ,  0.41942178,  0.14257312],
       [-0.05898739, -0.00745866, -0.04263731,  0.39894937,  0.14373579],
       [ 0.87751724,  0.11095761,  0.6342877 ,  0.31968675,  0.60489246],
       [ 0.70961842,  0.08972766,  0.51292695,  0.25851983,  0.09794085],
       [-0.03739011, -0.00472779, -0.02702635, -0.01362152, -0.00516055]])

In [142]:
k

array([[0.89300423, 0.29602646, 0.84324395, 0.50124763, 0.90828124],
       [0.98447369, 0.49777379, 0.804293  , 0.79612487, 0.74891846],
       [0.06929663, 0.06607341, 0.52658122, 0.45352045, 0.00623131],
       [0.08244464, 0.34976609, 0.92694995, 0.53735623, 0.46123932],
       [0.35770828, 0.41399092, 0.75771662, 0.43231041, 0.5996726 ],
       [0.11034219, 0.15365019, 0.57349453, 0.59646969, 0.6345068 ],
       [0.07967962, 0.71341071, 0.02267013, 0.73394101, 0.95380245]])

In [109]:


rndm = np.random.RandomState(1234)
b = rndm.uniform(size=(7, 5))
r2, st = QR(b)
q2 = Q_bilding(QR(b)[1])
assert_allclose(np.dot(q2, q2.T), np.eye(7), atol=1e-10)

assert_allclose(np.dot(q2, r2), b)

In [25]:
rndm = np.random.RandomState(4356)
b = rndm.uniform(size=(7, 5))
r2, st = QR(b)
q2 = Q_bilding(QR(b)[1])
assert_allclose(np.dot(q2, q2.T), np.eye(7), atol=1e-10)

assert_allclose(np.dot(q2, r2), b)

In [26]:
rndm = np.random.RandomState(823)
b = rndm.uniform(size=(7, 5))
r2, st = QR(b)
q2 = Q_bilding(QR(b)[1])
assert_allclose(np.dot(q2, q2.T), np.eye(7), atol=1e-10)

assert_allclose(np.dot(q2, r2), b)

Ну вроде я много на чем попробовал, кажется работает.